## Sequence to Sequence(seq2seq) and Attention

目前，最流行的seq2seq任务就是翻译，通常由一种自然语言翻译成另一种自然语言。当然除了自然语言互译外，还可以在编程语言之间进行翻译，更一般的是可以在任何标记的序列之间进行翻译，所以机器翻译可以指代任何一般的序列到序列的任务。

<img src="./images/examples.gif" alt="examples" style="zoom:50%;" />

接下来将分别介绍seq2seq的基本概念，以及密不可分的attention(注意力机制)，最后介绍最流行的模型-Transformer。

### Seq2Seq模型

所谓Seq2Seq(Sequence to Sequence)，即序列到序列模型，就是一种能够根据给定的序列，通过特定的生成方法生成另一个序列的方法，同时这两个序列可以不等长，这种结构又叫Encoder-Decoder模型，即编码-解码模型，是RNN的一个变种，为了解决RNN要求序列等长的问题。

通常，在机器翻译中任务中，给定一个输入序列$x_1$,$x_2$,...,$x_m$ 和输出序列$y_1$,$y_2$,...,$y_n$。翻译要做的就是在给定输入的情况下知道最有可能的目标序列，即得到最大概率可能出现的y：$p(y|x):y^* = \arg \max p(y|x)$。对于人类来所可以通过经验取推断出最有可能出现的y，但是在机器翻译中，我们需要通过学习带有参数$\theta$的函数$p(y|x,\theta)$来确定目标序列。

<img src="./images/human_machine_translation-min.png" alt="img" style="zoom: 25%;" />

#### Seq2Seq的应用场景

Seq2Seq的应用随着计算机科学与技术的发展，已经在很多领域产生了应用，如：

- 机器翻译：Seq2Seq最经典的应用，当前著名的Google翻译就是完全基于 Seq2Seq+Attention机制开发的。
- 文本摘要自动生成：输入一段文本，输出这段文本的摘要序列。
- 聊天机器人：第二个经典应用，输入一段文本，输出一段文本作为回答。
- 语音识别：输入语音信号序列，输出文本序列。
- 阅读理解：将输入的文章和问题分别编码，再对其解码得到问题的答案。
- 图片描述自动生成：自动生成图片的描述。
- 机器写诗歌，代码补全，故事风格改写，生成commit message等。

#### Encoder-Decoder 框架

Seq2Seq的基础结构由Encoder-Decoder组成，通常是由LSTM和GRU组成。编码器通过对输入序列进行学习，得到输入序列的特征表示$c$，然后将其作为解码器的输入来生成目标序列。

<img src="./images/image-20240127190338034.png" alt="image-20240127190338034" style="zoom: 50%;" />

#### 条件语言模型

对于经典的语言模型，主要是估计标记序列$y=(y_1,y_2,...,yn)$的概率$p(y)$，比如LSTM：

<img src="./images/image-20240127203435837.png" alt="image-20240127203435837" style="zoom:33%;" />

Seq2Seq作为经典的条件语言模型(Conditional Language Models)，与语言模型的区别在于还给定了一个source $x$作为输入，所以其训练过程与LMs很相似。

<img src="./images/lm_clm-min.png" alt="img" style="zoom:33%;" />

值得注意的是，CLMs不单单是解决Seq2Seq任务，在更通用的场景中，source $x$还可以是标记序列以外的东西，比如在图像描述任务(Image Captioning task)中，$x$可以是图片，而$y$是对图片的描述。

<img src="./images/50909ddc718d416cafccc1200cdd6d20.png" alt="50909ddc718d416cafccc1200cdd6d20" style="zoom: 50%;" />

因此，Seq2Seq的建模和训练过程与语言模型相似，pipeline的过程为：

- 将source $x$和前一个目标词汇喂入网络
- 从解码器中得到previous context(source和previous target tokens)的向量表示
- 根据向量表示预测下一个token的概率分布

<img src="./images/enc_dec_linear_out-min.png" alt="img" style="zoom: 25%;" />

#### 简单的Seq2Seq模型

<img src="./images/enc_dec_simple_rnn-min.png" alt="img" style="zoom: 33%;" />

最简单的Encoder-Decoder模型是由两个RNN(Lstm)组成，一个是Encoder，另一个是Decoder。Encoder Rnn读取输入序列，同时将最后一个时间步的state作为Decoder Rnn的初始化state。Encoder的最后一个状态包含输入序列的所有信息，这样Decoder可以基于这个状态向量生成目标序列。

上面的模型可以有一些改进的方式，比如编码器和解码器可以有多层，同时为了解决LSTM的长期依赖问题，比如在解码器在生成目标序列的开头时，可能已经忘记了与之最相关的早期输入的tokens，所以这里有一个trick可以有效解决这个问题：将source tokens进行翻转，target tokens不需要变，通过这种操作，模型就会产生很多的短期连接。

<img src="./images/reverse_order_before_after-min.png" alt="img" style="zoom:33%;" />

#### 训练

在训练过程中我们需要最大化模型分配给正确标记的概率，假设输入的训练序列为$x=(x_1,...,x_m)$，目标序列为$y=(y_1,...,y_n)$，在时间为$t$(timestep)时，模型预测的概率分布为$p^{(t)}=p(*|y_1,...,y_{t-1},x1,...,x_m)$。此时的target是$p^*=one-hot(y_t)$，也就是模型需要把概率1分配给正确的token,$y_t$,同时其他为0。

经典的损失函数就是交叉熵，目标分布$p^*$和预测分布$p$的loss为：
$$
Loss(p^*,p)=-p^*\log(p)=-\sum_{i=1}^{|V|}p_i^*\log(p_i).
$$
因为target为one-hot，所以只有一个位置($p_i^*$)为1，进一步：
$$
Loss(p^*,p)=-\log(p_{y_t})=-\log(p(y_t|y<t,x))
$$
也就是说在每一个timestep中会最大化模型分配给正确标记的概率：

<img src="./images/one_step_loss_intuition-min.png" alt="img" style="zoom: 33%;" />

最后是整个训练过程的图示：

<img src="./images/seq2seq_training_with_target.gif" alt="seq2seq_training_with_target" style="zoom:50%;" />

#### 推理：贪婪解码和束搜索

上面展示了模型的训练过程，但是在推理时我们需要的不是模型的原始输出即概率分布，而是从概率分布中选择一个最终的结果(一个单词或者是一系列单词)作为输出，也就是一个解码的过程。比如上面的用于机器翻译的Seq2Seq模型，期望最终输出的是I saw a cat on a mat，也就是找到近似解：

<img src="./images/inference_formula-min.png" alt="img" style="zoom: 25%;" />

常见的解码方式有贪婪解码和束搜索，他们有各自的优势和适用场景

> 贪婪解码(Greedy Decoding) - 每个时间步选择概率最高的词

最直接解码策略就是贪婪解码，也就是在每一个时间步生成时选择概率最高的那个词。这种方法实现简单，计算复杂度低，但是每个step的概率最高的词不一定就会组合成为最优的目标序列，也就是：

<img src="./images/greedy_is_bad-min.png" alt="img" style="zoom:25%;" />

> 束搜索(Beam Search) - 保留k个概率最高的词

不同于贪婪解码只保留一个词，束搜索会保留概率最高的k个可能的序列(束宽)。这这种方法虽然更复杂，但是保证了生成序列的多样性和生成的质量。束宽通常为4-10，增加束宽会降低计算效率，更重要的是会降低质量。

<img src="./images/beam_search.gif" alt="beam_search" style="zoom:50%;" />


### 构建简单的Seq2Seq架构

> 1. 构建语料库

In [1]:
# 1.训练数据 每一组数据包含输入序列、解码器输入以及目标输出(解码器的输入是Teacher Forcing)
sentences = [
    ['咖哥 喜欢 小冰', '<sos> KaGe likes XiaoBing', 'KaGe likes XiaoBing <eos>'],
    ['我 爱 学习 人工智能', '<sos> I love studying AI', 'I love studying AI <eos>'],
    ['深度学习 改变 世界', '<sos> DL changed the world', 'DL changed the world <eos>'],
    ['自然 语言 处理 很 强大', '<sos> NLP is so powerful', 'NLP is so powerful <eos>'],
    ['神经网络 非常 复杂', '<sos> Neural-Nets are complex', 'Neural-Nets are complex <eos>']]

# 2.初始化中英文词汇表
word_list_cn, word_list_en = [], []
for s in sentences:
    word_list_cn.extend(s[0].split())
    word_list_en.extend(s[1].split())
    word_list_en.extend(s[2].split())

# 对词汇表进行去重
word_list_cn = list(set(word_list_cn))
word_list_en = list(set(word_list_en))

# 分别构建单词到索引的映射
word2idx_cn = {word: idx for idx,word in enumerate(word_list_cn)}
word2idx_en = {word: idx for idx,word in enumerate(word_list_en)}

idx2word_cn = {idx: word for idx,word in enumerate(word_list_cn)}
idx2word_en = {idx: word for idx,word in enumerate(word_list_en)}

# 计算词汇表的大小
voc_size_cn = len(word_list_cn)
voc_size_en = len(word_list_en)
print(" 句子数量：", len(sentences)) # 打印句子数
print(" 中文词汇表大小：", voc_size_cn) # 打印中文词汇表大小
print(" 英文词汇表大小：", voc_size_en) # 打印英文词汇表大小
print(" 中文词汇到索引的字典：", word2idx_cn) # 打印中文词汇到索引的字典
print(" 英文词汇到索引的字典：", word2idx_en) # 打印英文词汇到索引的字典

 句子数量： 5
 中文词汇表大小： 18
 英文词汇表大小： 20
 中文词汇到索引的字典： {'语言': 0, '强大': 1, '改变': 2, '咖哥': 3, '喜欢': 4, '爱': 5, '学习': 6, '世界': 7, '我': 8, '小冰': 9, '处理': 10, '很': 11, '神经网络': 12, '非常': 13, '复杂': 14, '人工智能': 15, '深度学习': 16, '自然': 17}
 英文词汇到索引的字典： {'so': 0, 'world': 1, 'KaGe': 2, '<eos>': 3, 'likes': 4, 'love': 5, 'AI': 6, 'XiaoBing': 7, 'changed': 8, 'I': 9, 'are': 10, 'complex': 11, 'NLP': 12, 'the': 13, 'Neural-Nets': 14, 'studying': 15, 'is': 16, '<sos>': 17, 'powerful': 18, 'DL': 19}


> 2. 生成训练数据

In [2]:
import numpy as np
import torch
import random

# 定义生成训练数据的函数
# 之所以没有batch是因为输入序列的长度不一致，如果将会数据变成batch那么需要padding等操作保证输入序列长度一致
# 否则torch.LongTensor(encoder_input)会报错
def make_data(sentences):
    selected_sentence = random.choice(sentences)
    # np.array([[1,2,3]]) -> shape[1,3]多维数据, 行数可以看作是batch_size
    encoder_input = np.array([[word2idx_cn[s] for s in selected_sentence[0].split()]])
    decoder_input = np.array([[word2idx_en[s] for s in selected_sentence[1].split()]])
    target = np.array([[word2idx_en[s] for s in selected_sentence[2].split()]])

    encoder_input = torch.LongTensor(encoder_input)
    decoder_input = torch.LongTensor(decoder_input)
    target = torch.LongTensor(target)

    return encoder_input, decoder_input, target

encoder_input, decoder_input, target = make_data(sentences)

# 打印choice的句子
for s in sentences:
    cur = [word2idx_cn[n] in encoder_input for n in s[0].split()]
    if all(cur):
        orginal_sentence = s
        break

# 打印信息:
print(" 原始句子：", orginal_sentence) # 打印原始句子
print(" 编码器输入张量的形状：", encoder_input.shape)  # 打印输入张量形状
print(" 解码器输入张量的形状：", decoder_input.shape) # 打印输出张量形状
print(" 目标张量的形状：", target.shape) # 打印目标张量形状
print(" 编码器输入张量：", encoder_input) # 打印输入张量
print(" 解码器输入张量：", decoder_input) # 打印输出张量
print(" 目标张量：", target) # 打印目标张量
print(target.view(-1))
print(encoder_input.size(1))


 原始句子： ['咖哥 喜欢 小冰', '<sos> KaGe likes XiaoBing', 'KaGe likes XiaoBing <eos>']
 编码器输入张量的形状： torch.Size([1, 3])
 解码器输入张量的形状： torch.Size([1, 4])
 目标张量的形状： torch.Size([1, 4])
 编码器输入张量： tensor([[3, 4, 9]])
 解码器输入张量： tensor([[17,  2,  4,  7]])
 目标张量： tensor([[2, 4, 7, 3]])
tensor([2, 4, 7, 3])
3


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


> 3. 定义encoder和decoder

In [3]:
# 编码器
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, cn_input_size, embedding_size, hidden_size):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(cn_input_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, batch_first = True)
    def forward(self, input, hidden):
        emdedding_input = self.embedding(input)
        output, hn = self.rnn(emdedding_input, hidden)
        return output,hn

# 解码器
class Decoder(nn.Module):
    def __init__(self, en_input_size, embedding_size, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(en_input_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, batch_first = True)
        self.linear = nn.Linear(hidden_size, output_size)
    def forward(self, input, hidden):
        emdedding_input = self.embedding(input)
        output, hn = self.rnn(emdedding_input, hidden)
        output = self.linear(output)
        return output,hn

# 设置各个层的数量
cn_input_size = voc_size_cn
en_input_size = voc_size_en
embedding_size = 10
hidden_size = 128
output_size = voc_size_en

# 创建编码器和解码器
encoder = Encoder(cn_input_size, embedding_size, hidden_size)
decoder = Decoder(en_input_size, embedding_size, hidden_size, output_size)

print("编码器结构:", encoder)
print("解码器结构:", decoder)

编码器结构: Encoder(
  (embedding): Embedding(18, 10)
  (rnn): RNN(10, 128, batch_first=True)
)
解码器结构: Decoder(
  (embedding): Embedding(20, 10)
  (rnn): RNN(10, 128, batch_first=True)
  (linear): Linear(in_features=128, out_features=20, bias=True)
)


> 4. 定义Seq2Seq架构

In [4]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, cn_input, hidden, en_input):
        encoder_output, encoder_hn = self.encoder(cn_input, hidden)
        output, _ = self.decoder(en_input, encoder_hn)
        return output

model = Seq2Seq(encoder, decoder)
print("seq2seq:", model)

seq2seq: Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(18, 10)
    (rnn): RNN(10, 128, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(20, 10)
    (rnn): RNN(10, 128, batch_first=True)
    (linear): Linear(in_features=128, out_features=20, bias=True)
  )
)


> 5. 模型训练

In [5]:
# 对output和target进行view操作后再计算loss的必要性
output = torch.randn(2, 3, 20) # batch_size, n_step, vec_size
target = torch.randn(2, 3) # batch_size, n_step

# 相当于将所有batch的数据展开(堆叠), 然后计算loss
output = output.view(-1, 20) # (6, 20)
target = target.view(-1) # 6  

In [7]:
def train_seq2seq(model, criterion, optimizer, epochs):
    for epoch in range(epochs):
        encoder_input, decoder_input, target = make_data(sentences) # 创建训练数据
        h0 = torch.randn(1, encoder_input.size(0), hidden_size) # 初始化隐藏状态 
        optimizer.zero_grad() # 梯度清零
        output = model(encoder_input, h0, decoder_input)
        loss = criterion(output.view(-1, voc_size_en), target.view(-1)) # 计算损失函数
        if (epoch + 1) % 40 == 0:
            print(f"epoch: {epoch + 1:04d} cost: {loss:.6f}")
        loss.backward() # 反向传播
        optimizer.step() # 更新残暑
epochs = 400
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # 优化器
train_seq2seq(model, criterion, optimizer, epochs)

epoch: 0040 cost: 0.001701
epoch: 0080 cost: 0.000588
epoch: 0120 cost: 0.000388
epoch: 0160 cost: 0.000269
epoch: 0200 cost: 0.000208
epoch: 0240 cost: 0.000132
epoch: 0280 cost: 0.000105
epoch: 0320 cost: 0.000087
epoch: 0360 cost: 0.000080
epoch: 0400 cost: 0.000084


> 6.测试seq2seq架构

In [18]:
def test_seq2seq(model, source_sentence):
    encoder_input = np.array([[word2idx_cn[n] for n in source_sentence.split()]]) # 对sentence处理
    encoder_input = torch.LongTensor(encoder_input)
    # 构建输出的句子的索引，以 '<sos>' 开始，后面跟 '<eos>'，长度与输入句子相同
    decoder_input = np.array([word2idx_en['<sos>']] + [word2idx_en['<eos>']]*(len(encoder_input[0])-1))
    decoder_input = torch.LongTensor(decoder_input).unsqueeze(0)
    h0 = torch.zeros(1, encoder_input.size(0), hidden_size)
    predict = model(encoder_input, h0, decoder_input)
    # print(predict)
    res = predict.data.max(2, keepdim=True)[1] #获取概率最大对应的索引
    print(source_sentence, '->', [idx2word_en[n.item()] for n in res.squeeze()])
test_seq2seq(model, '自然 语言 处理 很 强大')


自然 语言 处理 很 强大 -> ['NLP', 'is', 'so', 'powerful', '<eos>']


### 小结  
Seq2Seq架构可以处理不等长的输入和输出序列, 因此在机器翻译和文本摘要等任务中表现出色。但是难以处理长序列(长输入序列可以导致信息损失)和复杂的上下文相关性等。具体来讲，在一个长序列中, 一些重要的上下文信息可能会被编码成一个固定长度的向量, 在解码过程中, 模型难以正确地关注到所有重要信息，因此也就引入了注意力机制-attention。